## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Import Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tuatapere,NZ,-46.1333,167.6833,52.83,72,96,7.70,overcast clouds
1,1,Touros,BR,-5.1989,-35.4608,77.90,82,92,7.18,overcast clouds
2,2,Karaton,ID,-6.3424,106.1045,86.95,59,100,5.82,overcast clouds
3,3,Jamestown,US,42.0970,-79.2353,38.08,65,75,9.22,broken clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,47.86,76,40,49.48,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature F you would like for your trip? "))
max_temp = float(input("What is the maxiumum temperature F you would like for your trip? "))

What is the minimum temperature F you would like for your trip? 40
What is the maxiumum temperature F you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
filtered_city_data_df.count()

City_ID                383
City                   383
Country                381
Lat                    383
Lng                    383
Max Temp               383
Humidity               383
Cloudiness             383
Wind Speed             383
Current Description    383
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = filtered_city_data_df.dropna()
clean_df.count()

City_ID                381
City                   381
Country                381
Lat                    381
Lng                    381
Max Temp               381
Humidity               381
Cloudiness             381
Wind Speed             381
Current Description    381
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tuatapere,NZ,52.83,overcast clouds,-46.1333,167.6833,
1,Touros,BR,77.90,overcast clouds,-5.1989,-35.4608,
4,Ushuaia,AR,47.86,scattered clouds,-54.8000,-68.3000,
7,Mount Gambier,AU,69.17,clear sky,-37.8333,140.7667,
10,Lagoa,PT,60.01,overcast clouds,39.0500,-27.9833,
11,Sao Joao Da Barra,BR,71.80,overcast clouds,-21.6403,-41.0511,
14,Iguape,BR,73.81,broken clouds,-24.7081,-47.5553,
17,Mataura,NZ,52.48,overcast clouds,-46.1927,168.8643,
18,Port Lincoln,AU,72.23,clear sky,-34.7333,135.8667,
21,Gombe,NG,75.42,overcast clouds,10.2897,11.1673,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

#Hotel Skipped counter
skipped_hotels = 0 

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    #add lat and lng location key for the params
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        skipped_hotels = skipped_hotels+ 1

print(f"Skipped {skipped_hotels} hotels.")

Skipped 33 hotels.


In [9]:
#Check Hotels Data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tuatapere,NZ,52.83,overcast clouds,-46.1333,167.6833,Ron and Tony's Bed & Breakfast
1,Touros,BR,77.90,overcast clouds,-5.1989,-35.4608,INN NEW HORIZON
4,Ushuaia,AR,47.86,scattered clouds,-54.8000,-68.3000,Albatross Hotel
7,Mount Gambier,AU,69.17,clear sky,-37.8333,140.7667,The Commodore
10,Lagoa,PT,60.01,overcast clouds,39.0500,-27.9833,Casa Das Faias


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the hotel_data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description & Max Temp </dt><dd>{Current Description} and max temp of {Max Temp}°F</dd>

</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))